In [61]:
from bs4 import BeautifulSoup
import requests
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeOptions
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [62]:
URL = "https://fibalivestats.dcd.shared.geniussports.com/u/ABE/2310604/bs.html#ASFSK"
page = requests.get(URL)
soup = BeautifulSoup(page.text, "html")

In [63]:
def flatten_and_append(data, new_list=None):
  if new_list is None:
    new_list = []
  
  for item in data:
    if isinstance(item, list):
      new_list = flatten_and_append(item, new_list)
    else:
      new_list.append(item)

  return new_list

In [64]:
def selenium_extractor_bench(URL):
    
    options = Options()
    options.add_argument("--headless") 
    driver = webdriver.Chrome(options=options)
    driver.get(URL)
    
    element_bench = driver.find_elements(By.CLASS_NAME, "bench")
    player_stats_bench = element_bench[0].text
    
    return player_stats_bench

In [65]:
def selenium_extractor_starters(URL):
    
    options = Options()
    options.add_argument("--headless") 
    driver = webdriver.Chrome(options=options)
    driver.get(URL)
    
    element_starters = driver.find_elements(By.TAG_NAME, "tbody")
    player_stats_starters = element_starters[1].text
    
    return player_stats_starters

In [66]:
def dictionary_creator(boxscore_section):

  player_stats_splitted = boxscore_section.split("\n")

  for i in range(0, len(player_stats_splitted), 3):
    player_stats_splitted[i+2] = player_stats_splitted[i+2].split(" ")
      
  flattened_list = flatten_and_append(player_stats_splitted)
  starters = {f"starter_{i + 1}": flattened_list[i:i+24] for i in range(0, len(flattened_list), 24)}

  return starters

In [67]:
boxscore = soup.find_all("table")[1]
column_names = []
headers = boxscore.find("thead").findAll("th")

for th in headers:
    column_names.append(th.text.strip())
column_names = column_names[:24]

In [68]:
player_stats_starters = dictionary_creator(selenium_extractor_starters("https://fibalivestats.dcd.shared.geniussports.com/u/ABE/2310604/bs.html#ASFSK"))
player_stats_bench = dictionary_creator(selenium_extractor_bench("https://fibalivestats.dcd.shared.geniussports.com/u/ABE/2310604/bs.html#ASFSK"))

In [69]:
print(player_stats_starters)
print(player_stats_bench)

{'starter_1': ['1', 'J. Sanchez', 'G', '32:23', '21', '7-11', '63', '5-7', '71', '2-4', '50', '5-8', '62', '0', '2', '2', '1', '1', '3', '0', '0', '2', '6', '19'], 'starter_25': ['10', 'J. Puente', 'PG', '23:51', '6', '1-3', '33', '0-2', '0', '1-1', '100', '3-5', '60', '0', '2', '2', '1', '3', '0', '0', '0', '5', '2', '22'], 'starter_49': ['11', 'J. Sanchez', 'G', '21:10', '7', '3-6', '50', '2-5', '40', '1-1', '100', '0-2', '0', '1', '0', '1', '1', '4', '2', '0', '0', '2', '2', '8'], 'starter_73': ['13', 'R. Flores', 'G', '28:00', '10', '2-10', '20', '2-9', '22', '0-1', '0', '6-6', '100', '1', '6', '7', '0', '3', '4', '0', '1', '2', '4', '22'], 'starter_97': ['37', 'R. De La Mora', 'F', '22:13', '10', '2-4', '50', '2-2', '100', '0-2', '0', '6-8', '75', '2', '7', '9', '1', '3', '0', '0', '0', '4', '5', '14']}
{'starter_1': ['3', 'M. Cardenas', 'G', '3:20', '0', '0-0', '0', '0-0', '0', '0-0', '0', '0-0', '0', '0', '1', '1', '0', '2', '0', '0', '0', '1', '0', '-1'], 'starter_25': ['9', 'C

In [72]:
import pandas as pd
df = pd.DataFrame(columns= column_names)

for key, value in player_stats_starters.items():
    df.loc[len(df)] = value
    
for key, value in player_stats_bench.items():
    df.loc[len(df)] = value

In [74]:
df = df.set_index("Jugador")

In [75]:
df

,No.,POS,MIN,Pt,TC,Ti%,2P,2P%,3P,3P%,...,RD,RT,AS,PER,REC,Tap,TapRe,FP,FR,+-Pt
Jugador,,,,,,,,,,,,,,,,,,,,,
J. Sanchez,1,G,32:23,21,7-11,63,5-7,71,2-4,50,...,2,2,1,1,3,0,0,2,6,19
J. Puente,10,PG,23:51,6,1-3,33,0-2,0,1-1,100,...,2,2,1,3,0,0,0,5,2,22
J. Sanchez,11,G,21:10,7,3-6,50,2-5,40,1-1,100,...,0,1,1,4,2,0,0,2,2,8
R. Flores,13,G,28:00,10,2-10,20,2-9,22,0-1,0,...,6,7,0,3,4,0,1,2,4,22
R. De La Mora,37,F,22:13,10,2-4,50,2-2,100,0-2,0,...,7,9,1,3,0,0,0,4,5,14
M. Cardenas,3,G,3:20,0,0-0,0,0-0,0,0-0,0,...,1,1,0,2,0,0,0,1,0,-1
C. Torres,9,G,14:59,3,1-5,20,1-3,33,0-2,0,...,0,1,1,1,0,0,2,0,1,-7
J. Michel,12,C,5:38,0,0-2,0,0-2,0,0-0,0,...,1,1,0,3,0,0,0,4,0,-8
L. Perez,15,G,7:37,0,0-2,0,0-2,0,0-0,0,...,1,3,0,0,0,0,0,1,0,-12
